## Средние значения метрик (по трем экспериментам)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv("se_blocks_infl.csv")

col1 = 'change4'
col2 = 'change4_ord'

# Prepare Data
colors = ['red'] * len(df)
colors2 = ['red'] * len(df)
indexes = df[df[col1]>0].index.tolist()
for i in range(len(indexes)):
    colors[indexes[i]] = 'green'
    
indexes = df[df[col2]>0].index.tolist()
for i in range(len(indexes)):
    colors2[indexes[i]] = 'green'
    
xs = np.arange(len(df))

labels = df["name"].tolist()
men_means = df[col1].tolist()
women_means = df[col2].tolist()

x = np.arange(len(labels))  # the label locations
width = 0.45  # the width of the bars

plt.rcParams.update({'font.size': 27})

fig, ax = plt.subplots(figsize=(23,11))
rects1 = ax.bar(x - width/2, men_means, label='Cross-Entropy Loss', edgecolor = "black", width=.45) # color=colors
rects2 = ax.bar(x + width/2, women_means, label='Ordinal Loss', edgecolor = "black", width=.45) # color=colors2

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xticks(x, labels,rotation='vertical')
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

plot_margin = 0.3
x0, x1, y0, y1 = plt.axis()
plt.axis((x0, x1, y0 - plot_margin, y1 + plot_margin))

fig.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv("ord_loss.csv")

col1 = 'KL-5'
col2 = 'KL-4'

# Prepare Data
colors = ['red'] * len(df)
colors2 = ['red'] * len(df)
indexes = df[df[col1]>0].index.tolist()
for i in range(len(indexes)):
    colors[indexes[i]] = 'green'
    
indexes = df[df[col2]>0].index.tolist()
for i in range(len(indexes)):
    colors2[indexes[i]] = 'green'
    
xs = np.arange(len(df))

labels = df["name"].tolist()
men_means = df[col1].tolist()
women_means = df[col2].tolist()

x = np.arange(len(labels))  # the label locations
width = 0.45  # the width of the bars

plt.rcParams.update({'font.size': 27})

fig, ax = plt.subplots(figsize=(37,17))
rects1 = ax.bar(x - width/2, men_means, label='KL-5', edgecolor = "black", width=.4) # color=colors
rects2 = ax.bar(x + width/2, women_means, label='KL-4', edgecolor = "black", width=.4) # color=colors2

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xticks(x, labels,rotation='vertical')
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

plot_margin = 0.25
x0, x1, y0, y1 = plt.axis()
plt.axis((x0, x1, y0 - plot_margin, y1 + plot_margin))

fig.tight_layout()

plt.show()

In [ ]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np

dirs = []
model_arch = 'SE_DenseNet121'
# working_dir = '2_grades\\'
# working_dir = '5_grades\\cross_entropy\\p\\'
working_dir = 'D:\\Development\\automatic-knee-oa-grading-tools\\cnn_converter\\'
# working_dir = 'F:\\Users\\Alexey\\Downloads\\Yandex.Disk.Files\\'
# working_dir = 'F:\\Users\\Alexey\\Downloads\\Yandex.Disk.Files\\ord\\'
for path in os.listdir(working_dir):
    if path.startswith(model_arch):
        dirs.append(path)
        
common_precision = 0.0
common_recall = 0.0
common_fscore = 0.0

max_precision = 0.0
max_recall = 0.0
max_fscore = 0.0

header = None
precision = None
recall = None
fscore = None

mats_num = 0
conf_mat = None

print("dirs:", dirs)
for arch in dirs:
    print('enter ' + arch + '...')
    
    # сренеклассовые метрики
    with open(working_dir + arch + '\\score_mean.txt') as f:
        line = f.readline()    
        vals = re.findall("\d+[.]\d+", line)   
        common_precision += float(vals[0])
        common_recall += float(vals[1])
        common_fscore += float(vals[3])

        max_precision = max(max_precision, float(vals[0]))
        max_recall = max(max_recall, float(vals[1]))
        max_fscore = max(max_fscore, float(vals[3]))
        
        # матрица ошибок
        ls = []
        line = f.readline()   
        mat_started = False
        while line:
            if line.startswith('[['):
                mat_started = True
                
            if mat_started:
                s = line.replace('[', '').replace(']', '').replace('\n', '').split(' ')
                s = [int(x) for x in s if len(x) > 0]
                ls.append(s)
                
            if line.endswith(']]'):
                break
            
            line = f.readline()  
    
        if conf_mat is None:
            conf_mat = np.matrix(ls)
            mats_num = 1.0
        else:
            conf_mat = conf_mat + np.matrix(ls)
            mats_num += 1.0
            
        print(np.matrix(ls))
    
    # метрики по каждому классу
    header1 = None
    recall1 = None
    fscore1 = None
    precision1 = None
    with open(working_dir + arch + '\\score.txt') as f:
        for line in f:
            if line.startswith('rows is') or '----' in line:
                continue
                
            vals = line[1:-2].replace(' ', '').split('|')
            if header1 is None:
                header1 = vals
                continue
                
            if precision1 is None:
                precision1 = dict(zip(header1, vals))
                continue
                
            if recall1 is None:
                recall1 = dict(zip(header1, vals))
                continue
                
            if fscore1 is None:
                fscore1 = dict(zip(header1, vals))
                continue
    
    if header is None:
        header = header1
        precision = dict.fromkeys(header, 0.0)
        recall = dict.fromkeys(header, 0.0)
        fscore = dict.fromkeys(header, 0.0)

    for k,v in precision1.items():
        precision[k] = precision[str(k)] + float(v)
    
    for k,v in recall1.items():
        recall[k] = recall[str(k)] + float(v)
        
    for k,v in fscore1.items():
        fscore[k] = fscore[str(k)] + float(v)
     
conf_mat = (conf_mat / mats_num).astype(int)
    
count = len(dirs)
precision = { k: v / count for k, v in precision.items()}
recall = { k: v / count for k, v in recall.items()}
fscore = { k: v / count for k, v in fscore.items()}

np.set_printoptions(precision=5)
print('   ')
print('   models:', dirs)
print('   -------')
print('   header:', np.array(list('  ' + k + '  ' for k in header)))
print('precision:', np.array(list(precision.values())))
print('   recall:', np.array(list(recall.values())))
print('   fscore:', np.array(list(fscore.values())))
print('   ')
print('max values:')
print('   -------')
print('precision:', '{:.2f}'.format(max_precision))
print('   recall:', '{:.2f}'.format(max_recall))
print('   fscore:', '{:.2f}'.format(max_fscore))
print('   ')
print('   avarage:')
print('   -------')
print('precision:', '{:.2f}'.format(common_precision / count))
print('   recall:', '{:.2f}'.format(common_recall / count))
print('   fscore:', '{:.2f}'.format(common_fscore / count))
print('   ')
print('latex row (avg):')
print('{:.2f}'.format(common_precision / count), '&',
      '{:.2f}'.format(common_recall / count), '&',
      '{:.2f}'.format(common_fscore / count))
print('   ')

# распечатаем матрицу ошибок
print('mean\nconfusion matrix:')
print('----------------')
print(conf_mat)

fig, ax = plt.subplots()
im = ax.imshow(conf_mat, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
    
plt.rcParams['figure.figsize'] = [11, 11]
plt.rcParams.update({'font.size': 27})

# We want to show all ticks...
ax.set(xticks=np.arange(conf_mat.shape[1]),
        yticks=np.arange(conf_mat.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=header, yticklabels=header,
        title='Confusion Matrix',
        ylabel='True label',
        xlabel='Predicted label')
    
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")
    
# Loop over data dimensions and create text annotations.
thresh = conf_mat.max() / 2.
for i in range(conf_mat.shape[0]):
    for j in range(conf_mat.shape[1]):
        ax.text(j, i, format(conf_mat[i, j], 'd'),
                ha="center", va="center",
                color="white" if conf_mat[i, j] > thresh else "black")
fig.tight_layout()
    